# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [28]:
#%pip install chart-studio
#%pip install cufflinks
#%pip install dash
#%pip install openpyxl
#%pip install --upgrade nbformat
#%pip install --upgrade jupyter nbconvert

     ------------------------------------- 256.6/256.6 kB 15.4 MB/s eta 0:00:00
     ---------------------------------------- 4.0/4.0 MB 25.5 MB/s eta 0:00:00
     ---------------------------------------- 123.4/123.4 kB ? eta 0:00:00
     ---------------------------------------- 162.8/162.8 kB ? eta 0:00:00
     ---------------------------------------- 48.0/48.0 kB 2.4 MB/s eta 0:00:00
     ------------------------------------- 377.9/377.9 kB 23.0 MB/s eta 0:00:00
     ---------------------------------------- 59.0/59.0 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 9.2/9.2 MB 36.7 MB/s eta 0:00:00
     ---------------------------------------- 93.5/93.5 kB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 83.1/83.1 kB ? eta 0:00:00
     ---------------------------------------- 61.5/61.5 kB ? eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 30.5 MB/s eta 0:00:00
     ---------------------------------------- 57.3/57.3 kB 3.1 MB/s

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [29]:
import pandas as pd
import chart_studio.plotly as py
import plotly.express as px
import cufflinks as cf
from ipywidgets import interact
import nbformat

cf.go_offline()

## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [19]:
df = pd.read_excel('../data/Online Retail.xlsx')

df.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


In [22]:
data = df[(df['Country'] != 'United Kingdom') & (df['InvoiceDate'] >= '2011-04-01') & (df['InvoiceDate'] <= '2011-04-30')]

data = data.groupby('Country').agg({'Quantity': 'sum', 'Revenue': 'sum'}).reset_index()

data.head()


,Country,Quantity,Revenue
0,Australia,224,421.60
1,Austria,308,584.78
2,Belgium,1170,1788.48
3,Brazil,356,1143.60
4,Channel Islands,96,243.00


In [30]:
data.iplot(kind='bar', xTitle='Country', title='Totales y Beneficios')
#me da error e instalandome todo lo que me pide sigue sin funcionar...

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [36]:
france=df[df.Country=='France']

france=france.loc[(france.InvoiceDate >= '2011-1-1') & (france.InvoiceDate <= '2011-5-31')]

france.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
807,551163,2011-04-26 15:52:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,9,2.95,26.55,12573,France
2645,540976,2011-01-12 15:00:00,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,3.75,15.00,12652,France
3290,545181,2011-02-28 14:45:00,22752,SET 7 BABUSHKA NESTING BOXES,4,8.50,34.00,12509,France
4614,542629,2011-01-31 09:57:00,84879,ASSORTED COLOUR BIRD ORNAMENT,160,1.45,232.00,12731,France
4795,548409,2011-03-31 10:27:00,84879,ASSORTED COLOUR BIRD ORNAMENT,160,1.45,232.00,12731,France


In [37]:
france = france[['Quantity', 'Revenue']]

france.head()

,Quantity,Revenue
807,9,26.55
2645,4,15.00
3290,4,34.00
4614,160,232.00
4795,160,232.00


In [38]:
france.iplot(kind='line', xTitle='Country', title='Totales y Beneficios')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [47]:
party = df.loc[df.Description=='PARTY BUNTING']
party = party.groupby('Country').mean(numeric_only=True)
party = party[['Quantity', 'UnitPrice']]
party.reset_index(inplace=True)
party.head()

,Country,Quantity,UnitPrice
0,Australia,33.125000,4.7125
1,Austria,8.000000,4.9500
2,Belgium,4.000000,4.9500
3,Channel Islands,13.333333,4.9500
4,Cyprus,2.333333,4.7500


In [49]:
party.iplot(x='Quantity', 
           y='UnitPrice', 
           categories='Country',
           xTitle='Average Quantity', 
           yTitle='Average unit price',
           color='red', 
           title='Average by Country')
;

AttributeError: module 'pandas' has no attribute 'np'

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [51]:
paises = ['EIRE', 'Germany', 'France', 'Netherlands']
filtro = df[(df['Country'].isin(paises)) & (df['Quantity'] > 0)]

fig = px.histogram(filtro, x='Quantity', color='Country', marginal='rug',
                   labels={'Quantity': 'Quantity per Invoice'},
                   title='Distribution of Quantity per Invoice by Country',
                   nbins=50, opacity=0.7)

fig.update_layout(barmode='overlay', showlegend=True)

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [ ]:
# hay que quitar uk xq es un outlier, xq es demasiado grande

In [53]:
plst=['JUMBO BAG RED RETROSPOT', 
          'CREAM HANGING HEART T-LIGHT HOLDER',
          'REGENCY CAKESTAND 3 TIER']

clst=['EIRE', 'Germany', 'France', 'Netherlands']

data=df.loc[df.Description.isin(plst)]
data=data.loc[data.Country.isin(clst)]

data=data.pivot_table(values='Revenue', 
                      columns='Description',
                      index='Country', 
                      aggfunc='sum')

data.reset_index(inplace=True)

data.iplot(kind='bar', 
           x='Country', 
           title='Revenue by Country',
           xTitle='Country', 
           yTitle='Revenue')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [55]:
import plotly.graph_objs as go

In [57]:
data6 = df[df['Country'] == 'United Kingdom']
data6['Day'] = pd.to_datetime(data6['InvoiceDate']).dt.day
data6['Month'] = pd.to_datetime(data6['InvoiceDate']).dt.month
data6['Year'] = pd.to_datetime(data6['InvoiceDate']).dt.year
data6 = data6.groupby(['Year', 'Month'])['Quantity'].sum().reset_index()

trace = go.Scatter(x=data6['Year'].astype(str) + '-' + data6['Month'].astype(str).str.zfill(2),
                   y=data6['Quantity'],
                   mode='lines',
                   name='Quantity Sold')
layout = go.Layout(title='Sales by Year-Month for the UK',
                   xaxis=dict(title='Date'),
                   yaxis=dict(title='Quantity'))
go.Figure(data=[trace], layout=layout)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'lines',
              'name': 'Quantity Sold',
              'type': 'scatter',
              'x': array(['2010-12', '2011-01', '2011-02', '2011-03', '2011-04', '2011-05',
                          '2011-06', '2011-07', '2011-08', '2011-09', '2011-10', '2011-11',
                          '2011-12'], dtype=object),
              'y': array([267569, 203995, 213291, 276218, 260346, 301803, 280931, 303500, 310791,
                          454016, 475727, 567319, 178791], dtype=int64)}],
    'layout': {'template': '...',
               'title': {'text': 'Sales by Year-Month for the UK'},
               'xaxis': {'title': {'text': 'Date'}},
               'yaxis': {'title': {'text': 'Quantity'}}}
})

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [59]:
unique_products_per_invoice = df.drop_duplicates(subset=['InvoiceNo', 'StockCode'])

product_stats = unique_products_per_invoice.groupby('StockCode').agg({'InvoiceNo': 'count', 'CustomerID': 'nunique'}).reset_index()

fig = px.scatter(product_stats, x='InvoiceNo', y='CustomerID', text='StockCode',
                 labels={'InvoiceNo': 'Number of Invoices', 'CustomerID': 'Number of Customers'},
                 title='Number of Invoices vs. Number of Customers per Product',
                 hover_name='StockCode')

fig.update_layout(
    sliders=[
        {'steps': [
            {'args': ['x', [0, 100]], 'label': '0-100'},
            {'args': ['x', [100, 200]], 'label': '100-200'},
            {'args': ['x', [200, 300]], 'label': '200-300'},
        ],
            'active': 0,
            'label': 'Number of Invoices Range',
            'method': 'relayout'},
        {'steps': [
            {'args': ['y', [0, 20]], 'label': '0-20'},
            {'args': ['y', [20, 40]], 'label': '20-40'},
            {'args': ['y', [40, 60]], 'label': '40-60'},
        ],
            'active': 0,
            'label': 'Number of Customers Range',
            'method': 'relayout'}
    ]
)

fig.show()

ValueError: Invalid property specified for object of type plotly.graph_objs.layout.Slider: 'label'

Did you mean "len"?

    Valid properties:
        active
            Determines which button (by index starting from 0) is
            considered active.
        activebgcolor
            Sets the background color of the slider grip while
            dragging.
        bgcolor
            Sets the background color of the slider.
        bordercolor
            Sets the color of the border enclosing the slider.
        borderwidth
            Sets the width (in px) of the border enclosing the
            slider.
        currentvalue
            :class:`plotly.graph_objects.layout.slider.Currentvalue
            ` instance or dict with compatible properties
        font
            Sets the font of the slider step labels.
        len
            Sets the length of the slider This measure excludes the
            padding of both ends. That is, the slider's length is
            this length 

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.

In [ ]:
def update_chart(description_filter):
    filtered_data = df[df['Description'].str.contains(description_filter, case=False)]
    
    fig = px.bar(
        filtered_data,
        x='Description',
        y='Revenue',
        text='Revenue',
        title='Revenue by Product Description',
        labels={'Description': 'Product Description', 'Revenue': 'Revenue'},
    )
    fig.update_traces(texttemplate='%{text:$,.2s}', textposition='outside')
    fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

    fig.show()

description_widget = widgets.Text(
    value='',
    description='Filter by Description:',
    continuous_update=False
)

widgets.interactive(update_chart, description_filter=description_widget)

update_chart("")  
display(description_widget)